<a href="https://colab.research.google.com/github/lydia0409/Planets/blob/master/Course_Work_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section 0:Preliminary Work**

## **Section 0.0:** Explore the given data

### **Section 0.01:** Check data first

Initial Setup and read Data

In [0]:
pip install -U skorch

In [0]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.ensemble as ens
import sklearn.svm as svm
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.model_selection import validation_curve
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')
#upload on Google Drive and mount the drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
# Read data from Google Drive
traing = pd.read_csv("/content/drive/My Drive/A50/train_set.csv")
valg = pd.read_csv("/content/drive/My Drive/A50/test_set.csv")

In [0]:
# check
traing.head()

Here we find that there is a column of unnamed variable.

In [0]:
# As 1st column is unknown parameters, we remove it
traing.drop(columns = 'Unnamed: 0', inplace=True)
valg.drop(columns = 'Unnamed: 0', inplace=True)

Check if there are zeros

In [0]:
# Check zeros in dataframe
(traing == 0).astype(int).sum()

In [0]:
# Check nans in dataframe
traing.isnull().sum()

In [0]:
traing.info()

In [0]:
#Check variables' formats and features
for i in range(0,7):
  print(traing[traing.columns[i]].value_counts())

Here we see that for some columns in dataframe, we have some variables in string format. Some of the columns have integer inputs. We need to convert those variables into categorical data in form of int64.


### **Section 0.02:** Histogram of data 

In [0]:
import matplotlib.pyplot as plt
# Plot histograms to check distribution of different variable
fig = plt.figure(figsize=(10,50 ))
for x in range(0,7):
      #fix the position of plot in subplots
      plt.subplot(7,1,x+1)
      #Plot hist
      plt.hist(traing[traing.columns[x]],bins=7)
      # add title
      plt.title(traing.columns[x])

Here we see that the data set is partially unbalanced, as number of 'unacc' data is much higher than the others.We need to use bootstrap to balance the dataset.


### **Section 0.03:** Bootstrap

**Bootstrap for train set**

In [0]:
# BootStrap
count_0, count_1, count_2, count_3 = traing.rating.value_counts()

# Divide by class
df_0 = traing[traing['rating'] == 'unacc']
df_1 = traing[traing['rating'] == 'acc']
df_2 = traing[traing['rating'] == 'good']
df_3 = traing[traing['rating'] == 'vgood']
# see balance
print(count_0, count_1, count_2, count_3)

In [0]:
# bootstrapping
df_0_over = df_0.sample(count_1, replace=True).reset_index().drop(columns=['index'])
df_1_over = df_1.sample(count_1, replace=True).reset_index().drop(columns=['index'])
df_2_over = df_2.sample(count_1, replace=True).reset_index().drop(columns=['index'])
df_3_over = df_3.sample(count_1, replace=True).reset_index().drop(columns=['index'])
# concatenating all classes again
traing = pd.concat([df_0_over, df_1_over, df_2_over, df_3_over])
traing = traing.reset_index()

**Bootstrap for test set**

In [0]:
count_01, count_11, count_21, count_31 = valg.rating.value_counts()
# Divide by class
df_01 = valg[valg['rating'] == 'unacc']
df_11 = valg[valg['rating'] == 'acc']
df_21 = valg[valg['rating'] == 'good']
df_31 = valg[valg['rating'] == 'vgood']
# see balance
print(count_01, count_11, count_21, count_31)

In [0]:
# bootstrapping
df_01_over = df_01.sample(count_11, replace=True).reset_index().drop(columns=['index'])
df_11_over = df_11.sample(count_11, replace=True).reset_index().drop(columns=['index'])
df_21_over = df_21.sample(count_11, replace=True).reset_index().drop(columns=['index'])
df_31_over = df_31.sample(count_11, replace=True).reset_index().drop(columns=['index'])
# concatenating all classes again
valg = pd.concat([df_01_over, df_11_over, df_21_over, df_31_over])
valg = valg.reset_index()

### **Section 0.04:** Encoder and Standardisation of the descriptors.

**Encoder**

In [0]:
#convert categorical string variables into numerical variables with values corresponding to strings
traing['buying1'] = traing['buying'].replace('low',0).replace('med',1).replace('high',2).replace('vhigh',3)
traing['maint1'] = traing['maint'].replace('low',0).replace('med',1).replace('high',2).replace('vhigh',3)
traing['doors1'] = traing['doors'].replace(2,0).replace(3,1).replace(4,2).replace('5more',3)
traing['persons1'] = traing['persons'].replace(2,0).replace(4,1).replace('more',2)
traing['lug_boot1'] = traing['lug_boot'] .replace('small',0).replace('med',1).replace('big',2)
traing['safety1'] = traing['safety'].replace('low',0).replace('med',1).replace('high',2)
traing['rating1'] = traing['rating'].replace('unacc',0).replace('acc',1).replace('good',2).replace('vgood',3)
#do the same thing for validation set 
valg['buying1'] = valg['buying'].replace('low',0).replace('med',1).replace('high',2).replace('vhigh',3)
valg['maint1'] = valg['maint'].replace('low',0).replace('med',1).replace('high',2).replace('vhigh',3)
valg['doors1'] = valg['doors'].replace(2,0).replace(3,1).replace(4,2).replace('5more',3)
valg['persons1'] = valg['persons'].replace(2,0).replace(4,1).replace('more',2)
valg['lug_boot1'] = valg['lug_boot'] .replace('small',0).replace('med',1).replace('big',2)
valg['safety1'] = valg['safety'].replace('low',0).replace('med',1).replace('high',2)
valg['rating1'] = valg['rating'].replace('unacc',0).replace('acc',1).replace('good',2).replace('vgood',3)


Replace commendatory string with high values and derogatory term with low values 

**Standardisation**

As the range of raw data may varies widely, some algorithms won't workd properly without standardization. In addition, the standardisation process will also make results converge much faster.

In [0]:
from sklearn import preprocessing
#standardisation before adding columns' name of training data
trainscalg = pd.DataFrame(preprocessing.scale(traing[['buying1','maint1','doors1','persons1','lug_boot1','safety1']]),columns = ['buying','maint','doors','persons','lug_boot','safety'])
#standardisation before adding columns' name of validation data
valscalg = pd.DataFrame(preprocessing.scale(valg[['buying1','maint1','doors1','persons1','lug_boot1','safety1']]),columns = ['buying','maint','doors','persons','lug_boot','safety'])
#add label column which donot need to be standardised
trainscalg['rating']=traing['rating1']
valscalg['rating']=valg['rating1']

In [0]:
#Split training and validation data
X_traing = trainscalg[['buying','maint','doors','persons','lug_boot','safety']]
Y_traing = trainscalg[['rating']]
X_valg = valscalg[['buying','maint','doors','persons','lug_boot','safety']]
Y_valg = valscalg[['rating']]

### **Section 0.05:**Try our data in simple RandomForest model and SVM model

In [0]:
# import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf_rf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf_rf.fit(X_traing,Y_traing)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:"+ str(metrics.accuracy_score( clf_rf.predict(X_traing), Y_traing)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score( clf_rf.predict(X_valg), Y_valg)*100)+'%')

In [0]:
# import SVM
from sklearn import svm
linear_svm = svm.SVC(kernel = 'linear')
linear_svm.fit(X_traing,Y_traing)
dec_linear = linear_svm.decision_function(X_traing)
linear_svm_pred_train = linear_svm.predict(X_traing)
linear_svm_pred_test = linear_svm.predict(X_valg)
# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:"+ str(metrics.accuracy_score(linear_svm_pred_train, Y_traing)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(linear_svm_pred_test, Y_valg)*100)+'%')

Here we see, the models trained on our dataset are extremely bad with very low accuracy. Also two models trained by given data shows significant underfitting which is not resonable. The given data may still have some problems other than unbalance after bootstrap which means it is not necessary to continue our work with given data.

### **Section 0.06:**Set CV to be stratified 5-fold 

In [0]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5 , shuffle =True)

## **Section 0.1:** Use the car data from UCI

https://archive.ics.uci.edu/ml/datasets/Car+Evaluation

### **Section 0.1.1:** Check Data

In [0]:
import pandas as pd
import numpy as np
# Read car data from Google Drive
car = pd.read_csv("/content/drive/My Drive/A50/car.csv")

In [0]:
#Plot histogram for rating variables
plt.hist(car['rating'],bins=7)
# add title
plt.title('rating')

Here we see, the new dataset is still unbalanced.We need to bootstrap dataset.

### **Section 0.1.2:** Bootstrap and split data into training set and validation set

In [0]:
# class count
count_0, count_1, count_2, count_3 = car.rating.value_counts()

# Divide by class
df_0 = car[car['rating'] == 'unacc']
df_1 = car[car['rating'] == 'acc']
df_2 = car[car['rating'] == 'good']
df_3 = car[car['rating'] == 'vgood']
# see balance
print(count_0, count_1, count_2, count_3)

In [0]:
# bootstrapping
df_0_over = df_0.sample(count_1, replace=True).reset_index().drop(columns=['index'])
df_1_over = df_1.sample(count_1, replace=True).reset_index().drop(columns=['index'])
df_2_over = df_2.sample(count_1, replace=True).reset_index().drop(columns=['index'])
df_3_over = df_3.sample(count_1, replace=True).reset_index().drop(columns=['index'])

In [0]:
# concatenating all classes again
car_over = pd.concat([df_0_over, df_1_over, df_2_over, df_3_over])

In [0]:
X_over = car_over[['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']]
y_over = car_over[['rating']]

In [0]:
#Split data into training set and validation set
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_over, y_over, test_size=0.2, random_state=0)

In [0]:
#Reset index
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
Y_train = Y_train.reset_index(drop=True)
Y_val = Y_val.reset_index(drop=True)

In [0]:
#Plot histogram for rating variables
plt.hist(Y_train['rating'],bins=7)
# add title
plt.title('rating')

In [0]:
#Plot histogram for rating variables
plt.hist(Y_val['rating'],bins=7)
# add title
plt.title('rating')

Here we find that both validation and training sets are balanced.

### **Section 0.1.3:** Encoder and Standardization

In [0]:
#Check variables' formats and features
for i in range(0,6):
  print(X_train[X_train.columns[i]].value_counts())

In [0]:
#convert categorical string variables into numerical variables with values corresponding to strings
X_train = X_train.replace('low',0).replace('med',1).replace('high',2).replace('vhigh',3)\
                .replace('5more',5).replace('more',6).replace('small',0).replace('big',2)
X_val = X_val.replace('low',0).replace('med',1).replace('high',2).replace('vhigh',3)\
                .replace('5more',5).replace('more',6).replace('small',0).replace('big',2)
Y_train = Y_train.replace('unacc',0).replace('acc',1).replace('good',2).replace('vgood',3)
Y_val = Y_val.replace('unacc',0).replace('acc',1).replace('good',2).replace('vgood',3)

In [0]:
#Check variables' formats and features
for i in range(0,6):
  print(X_val[X_val.columns[i]].value_counts())

**Standardisation**

As the range of raw data may varies widely, some algorithms won't workd properly without standardization. In addition, the standardisation process will also make results converge much faster.

In [0]:
#Here we only need to standardise X_train and X_val
from sklearn import preprocessing
X_train = pd.DataFrame(preprocessing.scale(X_train),columns = X_train.columns)
X_val = pd.DataFrame(preprocessing.scale(X_val),columns = X_train.columns)
#leave Y in form of int64

### **Section 0.1.4:** Train 2 simple models for our new training set

**RandomForestClassifier**

In [0]:
# import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train,Y_train)

# validation accuracy

rf_pred_train = clf_rf.predict(X_train)
rf_pred_val = clf_rf.predict(X_val)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:"+ str(metrics.accuracy_score(rf_pred_train, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(rf_pred_val, Y_val)*100)+'%')
from sklearn.metrics import confusion_matrix
confusion_matrix(rf_pred_val, Y_val)

Here we see that the RandomForest model fits new data much better than original data.

Linear SVM

In [0]:
# import SVM
from sklearn import svm
linear_svm = svm.SVC(kernel = 'linear')
linear_svm.fit(X_train,Y_train)
dec_linear = linear_svm.decision_function(X_train)
linear_svm_pred_train = linear_svm.predict(X_train)
linear_svm_pred_test = linear_svm.predict(X_val)
# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:"+ str(metrics.accuracy_score(linear_svm.predict(X_train), Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(linear_svm.predict(X_val), Y_val)*100)+'%')
confusion_matrix(rf_pred_val, Y_val)

Here we see that, although the accuracy of LinearSVM is obviously smaller than accuracy of RandomForest,the  Linear SVM model fits the new data much better than original data.

### **In the following CourseWork, car data will be used for further exploration**

# **Section 1:Random Forest Classifier**

## **Section 1.1:** Visualising different hyperparameters

### **Section 1.1.1:** Define validation curve function

Define function for detail plot of validation curve

In [0]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.model_selection import validation_curve
def visualhyper(train_score,test_score,hyper,param):

  # Calculate mean and standard deviation for training set scores
  train_mean = np.mean(train_score, axis=1)
  train_std = np.std(train_score, axis=1)

  # Calculate mean and standard deviation for test set scores
  test_mean = np.mean(test_scores, axis=1)
  test_std = np.std(test_scores, axis=1)

  # Plot mean accuracy scores for training and test sets
  plt.plot(param, train_mean, label="Training score", color="darkorange")
  plt.plot(param, test_mean, label="Cross-validation score", color="navy")

  # Plot accurancy bands for training and test sets
  plt.fill_between(param, train_mean - train_std, train_mean + train_std, color="darkorange",alpha=0.2)
  plt.fill_between(param, test_mean - test_std, test_mean + test_std, color="navy",alpha=0.2)
  # Create plot
  plt.title("Validation Curve Corresponding to {}".format(hyper))
  plt.xlabel(hyper)
  plt.ylabel("Accuracy Score")
  plt.legend(loc="best")
  plt.show()
  return [train_mean,test_mean];

define function for plot of general validation curve

In [0]:
def visualhyper1(train_score,test_score,hyper,param):
  train_scores_mean = np.mean(train_score, axis=1)
  train_scores_std = np.std(train_score, axis=1)
  test_scores_mean = np.mean(test_score, axis=1)
  test_scores_std = np.std(test_score, axis=1)

  plt.title("Validation Curve Corresponding to {}".format(hyper))
  plt.xlabel(hyper)
  plt.ylabel("Accuracy Score")
  plt.ylim(0.0, 1.1)
  lw = 2
  plt.semilogx(param, train_scores_mean, label="Training score",
              color="darkorange", lw=lw)
  plt.fill_between(param, train_scores_mean - train_scores_std,
                  train_scores_mean + train_scores_std, alpha=0.2,
                  color="darkorange", lw=lw)
  plt.semilogx(param, test_scores_mean, label="Cross-validation score",
              color="navy", lw=lw)
  plt.fill_between(param, test_scores_mean - test_scores_std,
                  test_scores_mean + test_scores_std, alpha=0.2,
                  color="navy", lw=lw)
  plt.legend(loc="best")
  plt.show()
  return [train_scores_mean,test_scores_mean];

### **Section 1.1.2:** Validation Curve for n_estimators

In [0]:
from sklearn.ensemble import RandomForestClassifier
# Create range of values for parameter
param_range = np.arange(1, 250, 10)
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(RandomForestClassifier(), 
                                             X_train, 
                                             Y_train, 
                                             param_name="n_estimators", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)


In [0]:
[trainacc,valacc] = visualhyper(train_scores,test_scores,'n_estimators',param_range)

In [0]:
 visualhyper1(train_scores,test_scores,'n_estimators',param_range)

The plot shows that the model is overfitting. As the increase of number of decision trees, the training accuracy reaches 100% and validation accuracy converges after n reaches 10 and fluctuate around. And the accuracy have maximum around n_estimators = 60. To see where it reaches maximum we can either plot a more detailed graph or find maximum in our pltacc(mean accuracy ).

There is no risk of overfitting in random forest with growing number of trees, as they are trained independently from each other. 

But for computing effeciency, we donot need large number of decision trees(n_estimators) as it's computational costly.

Also the shaded area decrease as variance decrease as n_estimators increase in the first plot which means the variance decreases.

Extract the param with higest accuracy and compare to our plots, we see that best number of trees should be around 50-70.

In [0]:
print('n_estimators corresponding to max accuracy for traing set:{}'.format(param_range[trainacc.argmax()]))
print('n_estimators corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

Check for a more detailed range:

In [0]:
param_range = np.arange(50,70, 5)
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(RandomForestClassifier(), 
                                             X_train, 
                                             Y_train, 
                                             param_name="n_estimators", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
[trainacc,valacc] = visualhyper(train_scores,test_scores,'n_estimators',param_range)

In [0]:
print('n_estimators corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

### **Section 1.1.3:** Validation Curve for max_depth

In [0]:
# Create range of values for parameter
param_range = np.arange(1, 40, 2)
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(RandomForestClassifier(), 
                                             X_train, 
                                             Y_train, 
                                             param_name="max_depth", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper1(train_scores,test_scores,'max_depth',param_range)

In [0]:
[trainacc,valacc] = visualhyper(train_scores,test_scores,'max_depth',param_range)

In [0]:
print('max_depth corresponding to max accuracy for traing set:{}'.format(param_range[trainacc.argmax()]))
print('max_depth corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

The plot shows that the model is overfitting. As the increase of max_depth, the training accuracy almost reaches 99.9% and validation accuracy converges and fluctuates around 98% after max_depth reaches 13. Also the accuracy for validation set is maximized around max_depth = 17. To see where it reaches maximum we can either plot a more detailed graph or find maximum in our pltacc(mean accuracy ). 

After max_depth reaches 13,  the deeper the tree, the more splits it has and it captures more information about the data and can be capable to deal with more complex problems. But it also increase the overfitting risk.In the plot, an important thing to notice is that, as the increase of max_depth, the gap between validation curve and training curve become wider which didn't show in validation curve for n_estimator.Which means the model is more overfitting as the increase of max_depth.

Also the shaded area decrease as variance decrease as max_depth increase in the first plot which means the variance decreases.


In addition, the increase of max_depth also increase computational cost. So we need to choose a suitable value of max_depth instead of focusing on accuracy only


In [0]:
param_range = np.arange(15, 30, 2)
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(RandomForestClassifier(), 
                                             X_train, 
                                             Y_train, 
                                             param_name="max_depth", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
[trainacc,valacc] = visualhyper(train_scores,test_scores,'n_estimators',param_range)

In [0]:
print('max_depth corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

### **Section 1.1.4:** Validation Curve for max_features

In [0]:
# Create range of values for parameter
param_range = ['auto','sqrt','log2']
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(RandomForestClassifier(), 
                                             X_train, 
                                             Y_train, 
                                             param_name="max_features", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'max_features',param_range)

The plot shows that the model is overfitting.Max_feature is the number of features to consider each time to make the split decision.
Increase of max_feature improve performance of model at each node, but also decreases the diversity of individual tree(Make trees similar) and increase the overfitting risk.

The plot shows overfitting pattern again. As we only have 6 features to consider, three methods provide similar max_feature which don't make obvious difference on accuracy.

However, 'auto' max_feature shows less overfitting pattern and highest accuracy in our graph. So 'auto' is the best choice in our case

### In conclusion, I think n_estimators and max_depth have bigger impact on performance. The accuracy curves of them change significantly when those two variables change. And max_depth has the greatest impact on the accuracy of our model.

## **Section 1.2:** Search for optimal hyperparameters

### **Section 1.2.1:** Train a simple RandomForestClassifier model

Train a simple model to see how RandomForestClassifier perform

In [0]:
# import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf_rf = RandomForestClassifier(n_estimators= 100, max_depth = 3)
clf_rf.fit(X_train,Y_train)

In [0]:
# check thee importances of all features
print(clf_rf.feature_importances_)

In [0]:
# validation accuracy
rf_pred_train = clf_rf.predict(X_train)
rf_pred_val = clf_rf.predict(X_val)

In [0]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:"+ str(metrics.accuracy_score(rf_pred_train, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(rf_pred_val, Y_val)*100)+'%')

The accuracy is not that good without a good choice of hyperparameter.

### **Section 1.2.2:** 5-fold Cross Validation by RandomizedSearchCV

We use RandomizedSearchCV to search for a large range and find specific parameter range for GridsearchCV

In [0]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 300, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
max_features.append(None)
# Maximum number of levels in tree
max_depth = np.arange(1, 60, 4)
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'bootstrap': bootstrap}

In [0]:
random_grid

Random Search Training

In [0]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                               n_iter=50,
                               scoring='accuracy', 
                               cv = skf, verbose=2, n_jobs=-1,
                               return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, Y_train);

In [0]:
rf_random.best_params_

Here the result is close to our conclusion in previous plotting part.As we only consider the accuracy, the max_depth and n_estimators will be much higher than the value it starts to converge

In [0]:
# prediction
rf_best = rf_random.best_estimator_
rf_pred_trainb = rf_best.predict(X_train)
rf_pred_valb = rf_best.predict(X_val)

In [0]:
# validation accuracy
print("Training Accuracy:"+ str(metrics.accuracy_score(rf_pred_trainb, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(rf_pred_valb, Y_val)*100)+'%')

The Accuracy of RandomForest trained with best_parameters generated by stratified-k-fold randomsearched reached 100% on training set and 99.35% on vavlidation set. 

### **Section 1.2.3:** 5-fold Cross Validation by GridSearchCV

In this task, we use the GridSearch to do a more detailed 5-fold cross validation on hyperparameters with range including the best_parameters found by RandomizedSearchCV.

In [0]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of grid search 
param_grid = {'n_estimators': [int(x) for x in np.linspace(start = 203, stop = 207, num = 5)],
               'max_features': ['auto', 'sqrt','log2'],
               'max_depth': [int(x) for x in np.linspace(35,39, num = 5)],
               'bootstrap': bootstrap}
# Create a based model
rf = RandomForestClassifier()
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                          cv = skf, n_jobs = -1, verbose = 2)

In [0]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train)
grid_search.best_params_

In [0]:
print('The optimal hyperparameters are:')
print(grid_search.best_params_)

In [0]:
# prediction
rf_best_grid = grid_search.best_estimator_
rf_pred_grid_trainb = rf_best_grid.predict(X_train)
rf_pred_grid_valb = rf_best_grid.predict(X_val)

In [0]:
# validation accuracy
print("Training Accuracy:"+ str(metrics.accuracy_score(rf_pred_grid_trainb, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(rf_pred_grid_valb, Y_val)*100)+'%')

As GridSearch range inculde the best parameter found by RandomizedSearch, the best_params found by GridSearch is our best_params.

The accuracy of result of GridSearch is the same as that of RandomSearch but slightly smaller. Because k-fold cross validation is a 'random' process, so its possible to have a relatively lower accuracy.

### The optimal hyperparameters are:
{'bootstrap': False, 'max_depth': 35, 'max_features': 'auto', 'n_estimators': 207}

# **Section 2:Support Vector Machines**

## **Section 2.1: Visualising different hyperparameters**

### **Section 2.1.1:** Validation Curve for C in linear SVM

As for linear kernel, we only have one hyperparamter in SVM:C

In [0]:
# Create range of values for parameter
param_range = [2**k for k in range(-5,5)]
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(svm.SVC(kernel = 'linear'), 
                                             X_train, 
                                             Y_train, 
                                             param_name="C", 
                                             param_range=param_range,
                                             cv=skf,
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'C',param_range)

In [0]:
visualhyper1(train_scores,test_scores,'C',param_range)

The plot shows that the model is overfitting.

Here, the Accuracy increases as the increase of C before C reaches 2 and converges afterward without any imporvements in accuracy.

But in that graph, accuracy stays around 89% with different values.That means the choice of different C does not make big difference(only increase of 5%). 

As second plot shows almost two straight lines with low accuracy, the linear model fit the data poorly compare to RandomForestClassifier.

Obviously, C is the most important hyperparameter in Linear SVM as its the only one and the change of c influence the accuracy of linear svm.

### **Section 2.1.2:** Validation Curve for C in poly SVM

In [0]:
# Create range of values for parameter
param_range = [2**k for k in range(-3,8)]
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(svm.SVC(kernel = 'poly'), 
                                             X_train, 
                                             Y_train, 
                                             param_name="C", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'C',param_range)


In [0]:
visualhyper1(train_scores,test_scores,'C',param_range)

The plot shows that the model is overfitting.

C is the penalty parameter of the error term. It controls the trade off between smooth decision boundary and classifying the training points correctly.

In the plot, the accuracy increase with the increase of penalty factor C. And the training curve converges to 100% as increase of C.

However,as the increase of C.Although the accuracy increase, it lead to an increase in variance, also the overfiting risk increases.In the plot we found that the gap between training score and validation score is wider as C increases. A large C gives you low bias and high variance. Low bias because you penalize the cost of missclasification a lot.
A small C gives you higher bias and lower variance.

So we need to find a balance between bias and variance.

### **Section 2.1.3:** Validation Curve for gamma in poly SVM

In [0]:
# Create range of values for parameter
param_range = [2**k for k in range(-3,8)]
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(svm.SVC(kernel = 'poly'), 
                                             X_train, 
                                             Y_train, 
                                             param_name="gamma", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'gamma',param_range)

In [0]:
[trainacc,valacc] = visualhyper1(train_scores,test_scores,'gamma',param_range)

In [0]:
print('gamma in poly model corresponding to max accuracy for traing set:{}'.format(param_range[trainacc.argmax()]))
print('gamma in poly model corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

The plot shows that the model is overfitting.

Gamma is the parameter of a Gaussian Kernel (to handle non-linear classification).
In the plot, the accuracy of training and validation increase as gamma increase until gamma reach 1 and 2 respectively. But as the gap between two curves is big and donot decrease, there is overfitting risk after gamma reaches a specific value between 1 and 2. 

Dont like C, large Gamma will cause high bias and low variance and viceversa. We need to find a balance between Variance and Bias for this model.
When gamma increase to a larage value, it will cause the decision hyperplane to overfit which will cause a high probability of missclassification for a new data(overfitting).But for the training line, increase of gamma will make decision hyperplane be more fitting to the training data.So the curve converges like in the graph.

### **Section 2.1.4:** Validation Curve for degree in poly SVM

In [0]:
# Create range of values for parameter
param_range = [1,2,3,4,5,6,7]
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(svm.SVC(kernel = 'poly'), 
                                             X_train, 
                                             Y_train, 
                                             param_name="degree", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'degree',param_range)

In [0]:
[trainacc,valacc] =visualhyper1(train_scores,test_scores,'degree',param_range)

In [0]:
print('degree in poly model corresponding to max accuracy for traing set:{}'.format(param_range[trainacc.argmax()]))
print('degree in poly model corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

The plot shows that the model is overfitting.

It’s basically the degree of the polynomial used to find the hyperplane to split the data. Higher degree allows a more flexible decision boundary and helps the SVM make an appropriate generalization, but as the degree increase to a specific level like 3 in our graph, the accuracy starts to decrease which means the SVM starts to overfit and the overfitting risk increases as the degree becomes higher and higher which cause a decrease in accuracy.

### **Conclusion for2.1.2&2.1.3&2.1.4:**

In poly svm model, all three hyperparameters are important and we need to find a balance between bias and variance. For all three hyperparameters, we need to find a point at which accuracy converges but the model is not quite overfitting. Which will be done in the next part, which using 5-fold crossvalidation to tune the optimal parameters.

### **Section 2.1.5:** Validation Curve for C in rbf SVM

In [0]:
# Create range of values for parameter
param_range = [2**k for k in range(-3,8)]
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(svm.SVC(kernel = 'rbf'), 
                                             X_train, 
                                             Y_train, 
                                             param_name="C", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'C',param_range)

In [0]:
[trainacc,valacc] =visualhyper1(train_scores,test_scores,'C',param_range)

In [0]:
print('C in rbf model corresponding to max accuracy for traing set:{}'.format(param_range[trainacc.argmax()]))
print('C in rbf model corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

The plot shows that the model is overfitting.

C is the penalty parameter of the error term. It controls the trade off between smooth decision boundary and classifying the training points correctly.

In the plot, the accuracy increase with the increase of penalty factor C. The training curve converges to 100% after C reaches 32.The validation curve converges after reaching maximum at 16.

However,as the increase of C.Although the accuracy increase, it lead to an increase in variance, the overfiting risk increases.In the plot we found that the gap between training score and validation score is wider as C increases. A large C gives low bias and high variance. Low bias because penalizing the cost of missclasification a lot. A small C gives higher bias and lower variance.

So we need to find a balance between bias and variance.

### **Section 2.1.6:** Validation Curve for gamma in rbf SVM

In [0]:
# Create range of values for parameter
param_range = [2**k for k in range(-3,8)]
skf = StratifiedKFold(n_splits = 5 , shuffle =True)
# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(svm.SVC(kernel = 'rbf'), 
                                             X_train, 
                                             Y_train, 
                                             param_name="gamma", 
                                             param_range=param_range,
                                             cv=skf, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [0]:
visualhyper(train_scores,test_scores,'gamma',param_range)

In [0]:
[trainacc,valacc] = visualhyper1(train_scores,test_scores,'gamma',param_range)

In [0]:
print('gamma in poly model corresponding to max accuracy for traing set:{}'.format(param_range[trainacc.argmax()]))
print('gamma in poly model corresponding to max accuracy for validation set:{}'.format(param_range[valacc.argmax()]))

The plot shows that the model is overfitting.

Gamma is the parameter of a Gaussian Kernel (to handle non-linear classification).
In the plot, the accuracy oincrease as gamma increase until gamma reach 1. But as the gap between two curves is bigger and bigger, there is overfitting risk afterwards.In the plot,the gap suddenly increases when gamma = 4 and converges to 80%.

Dont like C, large Gamma will cause high bias and low variance and viceversa. We need to find a balance between Variance and Bias for this model.
When gamma increase to a larage value, it will cause the decision hyperplane to overfit which will cause a high probability of missclassification for a new data(overfitting)so the accuracy decreases as gamma become large.But for the training line, increase of gamma will make decision hyperplane be more fitting to the training data.So the curve converges like in the graph.


### **Conclusion for2.1.5&2.1.6:**

In rbf svm model, both hyperparameters a and b are important and we need to find a balance between bias and variance. A large C gives low bias and high variance. Low bias because penalizing the cost of missclasification a lot. A small C gives higher bias and lower variance.By contrast,large Gamma will cause high bias and low variance and viceversa.
As I mentioned in poly svm, we can use 5-fold crossvalidation to tune the hyperparameters.

## **Section 2.2:** Search for optimal hyperparameters

###**Section 2.2.1:** Train Simple SVM Model

#### **Section 2.2.1.1:** Train a simple linear SVM Model


In [0]:
from sklearn import svm
#define linear svm model
linear_svm = svm.SVC(kernel = 'linear')
#train linear model with data
linear_svm.fit(X_train,Y_train)
#predict by model trained
linear_svm_pred_train = linear_svm.predict(X_train)
linear_svm_pred_test = linear_svm.predict(X_val)
#print accuracy output
print("Training Accuracy:"+ str(metrics.accuracy_score(linear_svm_pred_train, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(linear_svm_pred_test, Y_val)*100)+'%')

#### **Section 2.2.1.2:** Train a simple poly SVM Model

In [0]:
#define poly svm model
poly_svm = svm.SVC(kernel = 'poly')
#train poly svm model with data
poly_svm.fit(X_train,Y_train)
#predict by model trained
poly_svm_pred_train = poly_svm.predict(X_train)
poly_svm_pred_test = poly_svm.predict(X_val)
#print accuracy output
print("Training Accuracy:"+ str(metrics.accuracy_score(poly_svm_pred_train, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(poly_svm_pred_test, Y_val)*100)+'%')

#### **Section 2.2.1.3:** Train a simple rbf SVM Model

In [0]:
#define rbf svm model
rbf_svm = svm.SVC(kernel = 'rbf')
#train poly svm model with data
rbf_svm.fit(X_train,Y_train)
#predict by model trained
rbf_svm_pred_train = rbf_svm.predict(X_train)
rbf_svm_pred_test = rbf_svm.predict(X_val)
#print accuracy output
print("Training Accuracy:"+ str(metrics.accuracy_score(rbf_svm_pred_train, Y_train)*100)+'%')
print("Validation Accuracy:"+ str(metrics.accuracy_score(rbf_svm_pred_test, Y_val)*100)+'%')

For simple model, rbf has highest accuracy while the linear svm model performs poorly.

###**Section 2.2.2:** 5 fold Crossvalidation

#### **Section 2.2.2.1:** 5-fold Crossvalidation funciont

In [0]:
from sklearn.model_selection import RandomizedSearchCV
def Random_svm_hyper_search(X,Y,Xval,Yval,ker,Crange,gammarange,degreerange):
  # Create the random grid
  if gammarange == 'nan':
    randomgrid = {'C': Crange}
  elif degreerange == 'nan':
    randomgrid = {'C': Crange,
                  'gamma': gammarange}
  else:
    randomgrid = {'C': Crange,
                  'gamma': gammarange,
                  'degree': degreerange}
  svm_ran = svm.SVC(kernel = ker)
  # Random search of parameters, using 5 fold cross validation, 
  # search across 100 different combinations, and use all available cores
  skf = StratifiedKFold(n_splits = 5 , shuffle =True)
  svm_random = RandomizedSearchCV(estimator=svm_ran, param_distributions=randomgrid, cv = skf)
  svm_random.fit(X,Y)
  best_random = svm_random.best_estimator_
  print("Training Accuracy:"+ str(metrics.accuracy_score(best_random.predict(X), Y)*100)+'%')
  print("Validation Accuracy:"+ str(metrics.accuracy_score(best_random.predict(Xval), Yval)*100)+'%')
  return svm_random.best_params_

In [0]:
from sklearn.model_selection import GridSearchCV
def Grid_svm_hyper_search(X,Y,Xval,Yval,ker,Crange,gammarange,degreerange):
  # Create the gridSearch
  if gammarange == 'nan':
    grid = {'C': Crange}
  elif degreerange == 'nan':
    grid = {'C': Crange,
                  'gamma': gammarange}
  else:
    grid = {'C': Crange,
                  'gamma': gammarange,
                  'degree': degreerange}
  svm_ran = svm.SVC(kernel = ker)
  # Grid search of parameters, using 5 fold cross validation, 
  # search across 100 different combinations, and use all available cores
  skf = StratifiedKFold(n_splits = 5 , shuffle =True)
  svm_grid = GridSearchCV(estimator=svm_ran, param_grid=grid, cv = skf)
  svm_grid.fit(X,Y)
  best_grid = svm_grid.best_estimator_
  print("Training Accuracy:"+ str(metrics.accuracy_score(best_grid.predict(X), Y)*100)+'%')
  print("Validation Accuracy:"+ str(metrics.accuracy_score(best_grid.predict(Xval), Yval)*100)+'%')
  return svm_grid.best_params_

#### **Section 2.2.2.2:** 5-fold RandomizedSearchCV for linear SVM

In [0]:
C = [2**k for k in range(-4,8)]
#gamma is not considered in linear SVM
#degree is not considered in linear SVM
Random_svm_hyper_search(X_train,Y_train,X_val,Y_val,'linear',C,'nan','nan')

Here we have optimal parameter C with value 4 for linear SVM.

#### **Section 2.2.2.3:** 5-fold GridSearchCV for linear SVM

Use the result of RandomsearchCV to gridsearchCV through a more specific range

In [0]:
CG = [2.5,3,3.5,4,4.5,5,5.5]
#gamma is not considered in linear SVM
#degree is not considered in linear SVM
Grid_svm_hyper_search(X_train,Y_train,X_val,Y_val,'linear',CG,'nan','nan')

Here we have optimal parameter C with value 4 for linear SVM.The accuracy donot change a lot.

The result follows what we expected as in our validation plot. When C increase after reaching 4 , the accuracy donot improve and even decrease somehow. In conclusion, linear model donot perform as well as other model like RandomForest as the linear SVM assumption is the way too naive.

#### **Section 2.2.2.4:** 5-fold RandomizedSearchCV for poly SVM

In [0]:
C = [2**k for k in range(-3,6)]
gamma = [2**k for k in range(-3,6)]
degree = [1,2,3,4,5,6,7] 
Random_svm_hyper_search(X_train,Y_train,X_val,Y_val,'poly',C,gamma,degree)

By RandomizedSearchCV, we searched the optimal parameters which are C=1, degree=5, gamma=16. Define a new range including the best parameters found by RandomizedsearchCV.

#### **Section 2.2.2.5:** 5-fold GridSearchCV for poly SVM

Use the result of RandomsearchCV to gridsearchCV through a more specific range

In [0]:
CG = [0.25,0.5,0.75,1,1.25,1.5,1.75]
gammaG = [14,15,16,17,18]
degreeG = [3,4,5,6,7]
Grid_svm_hyper_search(X_train,Y_train,X_val,Y_val,'poly',CG,gammaG,degreeG)

The Training Accuracy didn't change, but the validation accuracy increase a lot by 4%. It shows quite good results with parameters provided by gridsearchcv.

So our optimal parameters are {'C': 0.25, 'degree': 3, 'gamma': 14}

#### **Section 2.2.2.6:** 5-fold RandomizedSearchCV for rbf SVM

In [0]:
C = [2**k for k in range(-3,5)]
gamma = np.logspace(-4,3,8)
#degree is not considered in linear SVM
Random_svm_hyper_search(X_train,Y_train,X_val,Y_val,'rbf',C,gamma,'nan')

By RandomizedSearchCV, we searched the optimal parameters which are C=1, gamma=1.0. Define a new range including the best parameters found by RandomizedsearchCV.

#### **Section 2.2.2.7:** 5-fold GridSearchCV for rbf SVM

Use the result of RandomsearchCV to gridsearchCV through a more specific range



In [0]:
CG = np.linspace(1.5,2.5,num=5)
gammaG = [0.7,0.8,0.9,1,1.1,1.2,1.3]
#degree is not considered in linear SVM
Grid_svm_hyper_search(X_train,Y_train,X_val,Y_val,'rbf',CG,gammaG,'nan')

The Training Accuracy and validation accuracy didn't change. It means the hyperparameters around above range donot influence accuracy a lot. 

So our optimal parameters are {'C': 1.5, 'gamma': 0.7}

###**Choice of kernel in SVM:** As in our 5-fold GridSearchCV process, we find that the rbf SVM have highest accuracy.


###**Choice SVM:** My choice of SVM model is: 'rbf' SVM with hyperparameters:{'C': 1.5, 'gamma': 0.7}

# **Section 3:Neural Networks**

In [0]:
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

## **Section 3.0:**Define Neural Network function

`input_size`: number of nodes in the input layer of the neural network;

`hidden_size`: number of neurons in each hidden layer;

`num_classes`: number of classes in MNIST;

`num_epochs`: number of epochs for training;

`batch_size`: number of examples the network sees before weights are updated;

`learning_rate`: the learning rate $\alpha$ in backpropagation;

`val_size`: 20% of the training data goes into the validation set.

As pytorch Neural Network donot accept Y in form of DataFrame, we change the format of Y into pandas.core.series.Series.

In [0]:
y_train = Y_train['rating']
y_val = Y_val['rating']

Define a funciton of neural network with input_size:6,hidden_size:200,num_classes:4,num_epochs:120 and chosen batchsize,learningrate.

In [0]:
def NNclassifier(Xtrain, Ytrain,  Xval, Yval, batchsize, learningrate):    
      input_size = 6  
      hidden_size = 200
      num_classes = 4
      num_epochs = 120
      batch_size = batchsize
      learning_rate = learningrate
      # prepare data
      xtrain = torch.from_numpy(Xtrain.values).float()
      ytrain = torch.from_numpy(np.array(Ytrain))
      xtest = torch.from_numpy(Xval.values).float()
      ytest = torch.from_numpy(np.array(Yval))
      # loading data 
      train = torch.utils.data.TensorDataset(xtrain, ytrain)
      train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size)
      test = torch.utils.data.TensorDataset(xtest, ytest)
      test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)
      class NeuralNet(nn.Module):
          def __init__(self, input_size, hidden_size, num_classes):
              super(NeuralNet, self).__init__()
              self.fc1 = nn.Linear(input_size, hidden_size)
              self.fc2 = nn.Linear(hidden_size, hidden_size)
              self.fc3 = nn.Linear(hidden_size, num_classes)
          
          def forward(self, x):
              out = F.relu(self.fc1(x))
              out = F.relu(self.fc2(out))
              out = self.fc3(out)
              return out

      net = NeuralNet(input_size, hidden_size, num_classes)
      # loss
      criterion = nn.CrossEntropyLoss()

      # optimiser
      optimiser = torch.optim.SGD(net.parameters(), lr=learning_rate)  
      total_step = len(train_loader)
      loss_values = []

      for epoch in range(num_epochs+1):
          ###################
          # train the model #
          ###################
        net.train()
        train_loss = 0.0  
        for i, (images, labels) in enumerate(train_loader, 0):
          images = images
          labels = labels
          # labels = labels.squeeze()    
          # forward pass
          outputs = net(images)
          loss = criterion(outputs, labels)
              
          # backward and optimise
          optimiser.zero_grad()
          loss.backward()
          optimiser.step()

          # update loss
          train_loss += loss.item()

          # print training statistics
          if (i+1) % 100 == 0:
            print('Epoch [{}/{}] \t Iteration [{}/{}] \t Training Loss: {:.6f}'.format(epoch, num_epochs, i+1, total_step, train_loss / 100))
            train_loss = 0.0
        
        loss_values.append(train_loss / 100)
      #Plotting loss over epochs:
      plt.figure(figsize=(12,8))
      plt.ylim(0,1)
      plt.xlim(-1, num_epochs+1)
      plt.ylabel('Training Loss', fontsize=20)
      plt.xlabel('Epoch', fontsize=20)
      plt.plot(loss_values)
      net.eval()
      
      #Print the accuracy of the test data
      correct = 0
      total = 0
      for images, labels in train_loader:
        images = images
        labels = labels
        # labels = labels.squeeze()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      print('Accuracy of the network on the train data: {} %'.format(100 * correct / total))

In [0]:
NNclassifier(X_train,y_train,X_val,y_val,64,0.01)

Here we see,we define batchsize=64,learningrate = 0.01. The training loss decreases as the the increase of epoch we have trained and finally converge to 0. The accuracy of the network is around 96.6%.

## **Section 3.1:**Try different learning rates

### **Section 3.1.1:**Learing Rate:0.0005

In [0]:
NNclassifier(X_train,y_train,X_val,y_val,64,0.0005)

### **Section 3.1.2:**Learing Rate:0.95

In [0]:
NNclassifier(X_train,y_train,X_val,y_val,64,0.95)

### **Conclusion of different linearning rate:**

The weights of a neural network cannot be calculated using an analytical method. Instead, the weights must be discovered via an empirical optimization procedure called stochastic gradient descent.As mentioned in lecture, we do it by differentiate the loss function with respect to weights and multiply it by the learning rate and add it to the initial weight.(It controls the amount of apportioned error that the weights of the model are updated with each time they are updated). Then updates the weights of the model using the back-propagation of errors algorithm.

Generally, Smaller learning rates require more training epochs given the smaller changes made to the weights each update, whereas larger learning rates result in rapid changes and require fewer training epochs for loss to converge. A learning rate that is too large can also cause a very quick convergence to suboptimal solution or fluctuate over training epochs and donot converge quickly.A learning rate is too small may never converge or converge to a local minimum but cannot 'jump out'.

In the first plot with learning rate 0.0005. We found that the loss converges in a very slow speed and stoped around 0.2 after 120 epochs. 120 epochs are not sufficient for the loss to converge to 0, for that learning rate, we need more epochs for loss function to converge which is computationally cost.
Also the accuracy is 74.75% which is quite low, because the loss isn't converged yet.

In the second plot with learning rate 0.95, luckly the loss converges without significant jump which could be the problem for large learning rate. The convergence speed is quite quick and the accuracy is 100%. The gradient is high initially as the rate of change of weight is updated with rate 0.95.

In the plot with learning rate 0.01, the curve is steeper than first plot and smooter than second plot. It shows a good convergence in loss.

To find best learning rate we can only do it numerically but not analytically.



## **Section 3.2:**Try different batch size

### **Section 3.2.1:**batch size:2

In [0]:
NNclassifier(X_train,y_train,X_val,y_val,2,0.01)

### **Section 3.2.2:**batch size:256

In [0]:
NNclassifier(X_train,y_train,X_val,y_val,256,0.01)

### **Conclusion of different batch size:**

Batch size is the number of training examples used in the estimate of the error gradient before each update of the weights of the model using the back-propagation of errors algorithm.It impacts how quickly a model learns and the stability of the learning process.

Smaller batch sizes are noisy, offering a regularizing effect and lower generalization error. A larger batch will carry out a significant degradation in the quality of the model, as measured by its ability to generalize.

In the plot of batch size 2, the function provide an output of training process. In which , the training accuracy tends to 0 after iteration of 20 epochs and the final accuracy is 100%.The plot converges quickly.However, it is computationally cost as it require long time to train the model.

In the plot of batch size 256, the runtime of our funtion is much shorter than that of size2.The accuracy is only 89% and the plot of loss converges much slower than plot of batch size 2. The plot shows a flat line as high batch size means low updating frequency such that for each epoch, weight is updated less times like twice or once each epoch, such that the loss decrease slowly as epoch increase.

## **Section 3.3:**Dropout regularisation by selfdefined function


### **Section 3.3.0:**Function of Neural Network with dropout in second hidden layer

In [0]:
def NNclassifierdrop(Xtrain, Ytrain,  Xval, Yval, batchsize, learningrate,dropoutrate):    
      input_size = 6  
      hidden_size = 200 
      num_classes = 4
      num_epochs = 120
      batch_size = batchsize
      learning_rate = learningrate
      # prepare data
      X_train = torch.from_numpy(Xtrain.values).float()
      y_train = torch.from_numpy(np.array(Ytrain))
      X_test = torch.from_numpy(Xval.values).float()
      y_test = torch.from_numpy(np.array(Yval))
      # loading data 
      train = torch.utils.data.TensorDataset(X_train, y_train)
      train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size)
      test = torch.utils.data.TensorDataset(X_test, y_test)
      test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)
      class NeuralNet(nn.Module):
          def __init__(self, input_size, hidden_size, num_classes):
              super(NeuralNet, self).__init__()
              self.fc1 = nn.Linear(input_size, hidden_size)
              self.fc2 = nn.Linear(hidden_size, hidden_size)
              self.dropout = torch.nn.Dropout(p= dropoutrate)
              #p: probability of an element to be zeroed
              self.fc3 = nn.Linear(hidden_size, num_classes)
          def forward(self, x):
              out = F.relu(self.fc1(x))
              out = F.relu(self.fc2(out))
              out = self.dropout(out)
              out = self.fc3(out)
              return out

      net = NeuralNet(input_size, hidden_size, num_classes)
      # loss
      criterion = nn.CrossEntropyLoss()

      # optimiser
      optimiser = torch.optim.SGD(net.parameters(), lr=learning_rate)  
      total_step = len(train_loader)
      loss_values = []

      for epoch in range(num_epochs+1):
        net.train()
        train_loss = 0.0  
        for i, (images, labels) in enumerate(train_loader, 0):
          images = images
          labels = labels
              
          # forward pass
          outputs = net(images)
          loss = criterion(outputs, labels)
              
          # backward and optimise
          optimiser.zero_grad()
          loss.backward()
          optimiser.step()

          # update loss
          train_loss += loss.item()

          # print training statistics
          if (i+1) % 100 == 0:
            print('Epoch [{}/{}] \t Iteration [{}/{}] \t Training Loss: {:.6f}'.format(epoch, num_epochs, i+1, total_step, train_loss / 100))
            train_loss = 0.0
        
        loss_values.append(train_loss / 100)
      #Print the accuracy of the test data
      net.eval()
      correct = 0
      total = 0
      for images, labels in test_loader:
        images = images
        labels = labels
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      accuracy = correct/total
      return accuracy

### **Section 3.3.1:**Develop a selfdefined 5-fold crossvalidation method

In [0]:
from sklearn.model_selection import StratifiedKFold
#Define stratifiedKFold
skf = StratifiedKFold(n_splits=5 , shuffle = True)
#Define index for K-FOLD
k = 0
#Define P choice
P = [(i)*0.1 for i in np.arange(0,11)]
#p is the length of P
p = len(P)
#Define an Acc array to accept calculated data
Acc = np.zeros((5,p))

for train_index, test_index in skf.split(X_train,Y_train):
  #Split traning and validation set in stratifiedKFold
  train_x , test_x = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
  train_y , test_y = Y_train.iloc[train_index,:],Y_train.iloc[test_index,:]
  for i in np.arange(p):
    #Use defined function to calculate related accuracy
    Acc[k][i] = NNclassifierdrop(train_x,train_y['rating'],test_x,test_y['rating'],64,0.01,P[i])
  k=k+1

In [0]:
# Convert Accuracy into a dataframe
Accdf = pd.DataFrame(Acc)

In [0]:
# Derive the mean accuracy of 5-fold stratified cross validation
ACC = pd.DataFrame(Accdf.mean(axis = 0), columns = ['Accuracy'])

In [0]:
#Here the ACC represents the accuracy output corresponding to dropout rate in P provided by 5-fold stratified crossvalidation
ACC

In [0]:
print('Dropout rate with highest accuracy:{}'.format(P[int(ACC.idxmax(axis = 0))]))

In [0]:
plt.plot(P,ACC)
plt.title('Accuracy dropoutrate Plot')
plt.xlabel('Dropout Rate')
plt.ylabel('Accuracy')

Here we see, as the increase of dropout rate, the Accuracy decrease significantly and drop to 0,3 after dropoutrate increase to 0.9. Here the dropoutrate is the probability at which outputs of the layer are dropped out.As the increase of p, less and less neurons are trained in a layer. As lowe dropout rates(0%-40%) has similar accuracy in our case. We say the number of neurons(200) defined is good in our case.

An important use of dropoutrate is to solve the problem of overfitting, the plot shows that our design of hidden layer is good which donot have an obvious overfitting pattern.

### **Section 3.3.2:**Dropout regularisation by selfdefined function

In [0]:
def dropout_kfold_cv(Xtrain, Ytrain,  Xval, Yval,k,P):
  #Define stratifiedKFold
  skf = StratifiedKFold(n_splits=k , shuffle = True)
  #Define index for K-FOLD
  j = 0
  #p is the length of P
  p = len(P)
  #Define an Acc array to accept calculated data
  Acc = np.zeros((k,p))
  for train_index, test_index in skf.split(Xtrain,Ytrain):
    #Split traning and validation set in stratifiedKFold
    train_x , test_x = Xtrain.iloc[train_index,:],Xtrain.iloc[test_index,:]
    train_y , test_y = Ytrain.iloc[train_index,:],Ytrain.iloc[test_index,:]
    for i in np.arange(p):
      #Use defined function to calculate related accuracy
      Acc[j][i] = NNclassifierdrop(Xtrain,Ytrain['rating'],Xval,Yval['rating'],64,0.01,P[i])
    j=j+1
  # Convert Accuracy into a dataframe
  Accd = pd.DataFrame(Acc)
  # Derive the mean accuracy of 5-fold stratified cross validation
  ACC = pd.DataFrame(Accd.mean(axis = 0), columns = ['Accuracy'])
  #Here the ACC represents the accuracy output corresponding to dropout rate in P provided by 5-fold stratified crossvalidation
  optimalp = P[int(ACC.idxmax(axis = 0))]
  return [ACC,optimalp]

In [0]:
# [a,b] = dropout_kfold_cv(X_train, Y_train,  X_val, Y_val,5,[(i)*0.05 for i in np.arange(0,21)])

In [0]:
# plt.plot(P,a)
# plt.title('Accuracy dropoutrate Plot')
# plt.xlabel('Dropout Rate')
# plt.ylabel('Accuracy')

# **Section 4:Discussion**

## **Section 4.1:**Compare performance of the three classifier

Recall is defined as the number of relevant documents retrieved by a search divided by the total number of existing relevant documents.

precision is defined as the number of relevant documents retrieved by a search divided by the total number of documents retrieved by that search.

F1 score is the harmonic mean of the precision and recall

F1 = 2*precision*recall/(precision+recall)

Precision = True Positive/(True positive +False positive)

Recall = True Positive/(True Posive+False Negative)


### **Section 4.1.1:**RandomForestClassifier

The optimal hyperparameters are:
{'bootstrap': False, 'max_depth': 35, 'max_features': 'auto', 'n_estimators': 207}

In [0]:
# import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
clf_rf_optimal = RandomForestClassifier(n_estimators= 207, max_depth = 35,max_features = 'auto',bootstrap='False')
clf_rf_optimal.fit(X_train,Y_train) 

In [0]:
#Make prediction
rf_optimal_pred = clf_rf_optimal.predict(X_val)

In [0]:
print('The validation accuracy of Optimal RandomForestClassifier:{}'.format(metrics.accuracy_score(rf_optimal_pred,Y_val)*100)+'%')

Find the confusion matrix for RandomForest model

In [0]:
rf_conf = confusion_matrix(rf_optimal_pred, Y_val)
print(rf_conf)

In [0]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

ax= plt.subplot()
sns.heatmap(rf_conf, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix for optimal RF'); 
ax.xaxis.set_ticklabels(['unacc','acc','good','vgood']); ax.yaxis.set_ticklabels(['unacc','acc','good','vgood']);
plt.show()

Here we see that the numbers on diagonal is big and others are almost 0. Which means our optimal RandomForestClassifier performs quite well.

In [0]:
Accuracy_rf = metrics.accuracy_score(rf_optimal_pred, Y_val)
Precision_rf = metrics.precision_score(rf_optimal_pred, Y_val,average='macro')
Recall_rf = metrics.recall_score(rf_optimal_pred, Y_val,average='macro')
F1_rf = metrics.f1_score(rf_optimal_pred, Y_val,average='macro')

In [0]:
print('The validation Accuracy of our optimal RF:{}'.format(Accuracy_rf*100)+'%')
print('The validation Precision Score of our optimal RF:{}'.format(Precision_rf*100)+'%')
print('The validation Recall of our optimal RF:{}'.format(Recall_rf*100)+'%')
print('The validation F1-score of our optimal RF:{}'.format(F1_rf*100)+'%')

This shows overfitting problem as validation accuracy is lower than training accuracy trained before.

As we see that for Accuracy,Precision,recall and F1.All those values are similar.

Precision and Recall is a measure of quality of classification to see if missclassification exists.F1 Score can be see as the combination of precision and recall. As we calculated the Precision and Recall, F1 Score seens to be repeating.

In this model, the model fits data quiet well and produce almost perfect classification. So we have similar values on above measures. We only need to consider the Accuracy.

### **Section 4.1.2:**SVM optimal model

Choice SVM: My choice of SVM model is: 'rbf' SVM with hyperparameters:{'C': 1.5, 'gamma': 0.7}

In [0]:
#define rbf svm model
clf_svm_opt = svm.SVC(kernel = 'rbf',C=1.5,gamma=0.7)
#train poly svm model with data
clf_svm_opt.fit(X_train,Y_train)
#predict by model trained
clf_svm_opt_pred = clf_svm_opt.predict(X_val)
#print accuracy output
print("Validation Accuracy:"+ str(metrics.accuracy_score(clf_svm_opt_pred, Y_val)*100)+'%')

In [0]:
svm_conf = confusion_matrix(clf_svm_opt_pred, Y_val)
print(svm_conf)

In [0]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

ax= plt.subplot()
sns.heatmap(svm_conf, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix for optimal SVM'); 
ax.xaxis.set_ticklabels(['unacc','acc','good','vgood']); ax.yaxis.set_ticklabels(['unacc','acc','good','vgood']);
plt.show()

Here we see that the numbers on diagonal is big and others are almost 0. Which means our optimal RandomForestClassifier performs quite well.

In [0]:
Accuracy_svm = metrics.accuracy_score(clf_svm_opt_pred, Y_val)
Precision_svm = metrics.precision_score(clf_svm_opt_pred, Y_val,average='macro')
Recall_svm = metrics.recall_score(clf_svm_opt_pred, Y_val,average='macro')
F1_svm = metrics.f1_score(clf_svm_opt_pred, Y_val,average='macro')

In [0]:
print('The validation Accuracy of our optimal SVM:{}'.format(Accuracy_svm*100)+'%')
print('The validation Precision Score of our optimal SVM:{}'.format(Precision_svm*100)+'%')
print('The validation Recall of our optimal SVM:{}'.format(Recall_svm*100)+'%')
print('The validation F1-score of our optimal SVMF:{}'.format(F1_svm*100)+'%')

This shows overfitting problem as validation accuracy is lower than training accuracy trained before.

As we see that for Accuracy,Precision,recall and F1.All those values are similar and slightly lower than those of Optimal SVM model.

As mentioned before, Precision and Recall is a measure of quality of classification to see if missclassification exists.F1 Score can be see as the combination of precision and recall. As we calculated the Precision and Recall, F1 Score seens to be repeating.

In this model, the model fits data quiet well and produce almost perfect classification. So we have similar values on above measures. We only need to consider the Accuracy.

### **Section 4.1.3:**Neural optimal model with dropoutrate = 0

In [0]:
input_size = 6  
hidden_size = 200 
num_classes = 4
num_epochs = 120
batch_size = 64
learning_rate = 0.01
# prepare data
X_train1 = torch.from_numpy(X_train.values).float()
y_train1 = torch.from_numpy(np.array(y_train))
X_test1 = torch.from_numpy(X_val.values).float()
y_test1 = torch.from_numpy(np.array(y_val))
# loading data 
train = torch.utils.data.TensorDataset(X_train1, y_train1)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size)
test = torch.utils.data.TensorDataset(X_test1, y_test1)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.dropout = torch.nn.Dropout(p= 0)
        #p: probability of an element to be zeroed
        self.fc3 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.fc3(out)
        return out

net = NeuralNet(input_size, hidden_size, num_classes)
# loss
criterion = nn.CrossEntropyLoss()

# optimiser
optimiser = torch.optim.SGD(net.parameters(), lr=learning_rate)  
total_step = len(train_loader)
loss_values = []

for epoch in range(num_epochs+1):
  net.train()
  train_loss = 0.0  
  for i, (images, labels) in enumerate(train_loader, 0):
    images = images
    labels = labels
        
    # forward pass
    outputs = net(images)
    loss = criterion(outputs, labels)
        
    # backward and optimise
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    # update loss
    train_loss += loss.item()

    # print training statistics
    if (i+1) % 100 == 0:
      print('Epoch [{}/{}] \t Iteration [{}/{}] \t Training Loss: {:.6f}'.format(epoch, num_epochs, i+1, total_step, train_loss / 100))
      train_loss = 0.0
  
  loss_values.append(train_loss / 100)
#Print the accuracy of the test data

In [0]:
net.eval()

validation_nn = []
predict_nn = []
for images, labels in test_loader:
  images = images
  labels = labels
  outputs = net(images)
  _, predicted = torch.max(outputs.data, 1)
  predicted_array = predicted.numpy()
  validation_array = labels.numpy()
  validation_nn = np.append(validation_nn,validation_array)
  predict_nn = np.append(predict_nn,predicted_array)

Here we have our prediction and validation set, so we can compute the confusion matrix

In [0]:
nn_conf = confusion_matrix(predict_nn, validation_nn)
print(nn_conf)

In [0]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

ax= plt.subplot()
sns.heatmap(nn_conf, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix for NN '); 
ax.xaxis.set_ticklabels(['unacc','acc','good','vgood']); ax.yaxis.set_ticklabels(['unacc','acc','good','vgood']);
plt.show()

Here we see that the numbers on diagonal is big and others are almost 0. Which means our optimal RandomForestClassifier performs quite well. But the missclassification in this model is more than obvious 2 models.

In [0]:
Accuracy_nn = metrics.accuracy_score(predict_nn, validation_nn)
Precision_nn = metrics.precision_score(predict_nn, validation_nn,average='macro')
Recall_nn = metrics.recall_score(predict_nn, validation_nn,average='macro')
F1_nn = metrics.f1_score(predict_nn, validation_nn,average='macro')

In [0]:
print('The validation Accuracy of our optimal NN:{}'.format(Accuracy_nn*100)+'%')
print('The validation Precision Score of our optimal NN:{}'.format(Precision_nn*100)+'%')
print('The validation Recall of our optimal NN:{}'.format(Recall_nn*100)+'%')
print('The validation F1-score of our optimal NN:{}'.format(F1_nn*100)+'%')

The neural network model seens to have lowest accuracy and perform worst in above measures.

This shows overfitting problem as validation accuracy is lower than training accuracy trained before.


As we see that for Accuracy,Precision,recall and F1.All those values are similar and slightly lower than those of Optimal SVM model.

As mentioned before, Precision and Recall is a measure of quality of classification to see if missclassification exists.F1 Score can be see as the combination of precision and recall. As we calculated the Precision and Recall, F1 Score seens to be repeating.

In this model, the model fits data quiet well and produce almost perfect classification. So we have similar values on above measures. We only need to consider the Accuracy.

## **Section 4.2:**Choice of our best Model.

Here, we only use accuracy to measure if a model is our best choice as other measures like Precision, Recall and F1 shows similar values and properties for our model with very fitting data.

As we donot have crossvalidation on different parameters in NeuralNetwork model. It performs worst in our case, so wo won't analyse it here.

For optimal RFC:

Training Accuracy:100.0%
Validation Accuracy:99.02597402597402%

For optimal SVM:

Training Accuracy:100.0%
Validation Accuracy:98.37662337662337%

Both RFC and SVM have 100% accuracies on training set.

As RFC is designed to solve the problem of overfitting.

We see the optimal RFC have least overfitting problem, which means it fits new data better.

Also the RFC has less parameters and is easier to implement, by changing tree size and tree number.

Compare to other 2 models, it also cost less computationally.